# ===============================================================
# Shopee Seasonal Product Analysis - Full Preprocessing Pipeline
# ===============================================================

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

# ================================
# 1. Load Dataset
# ================================

In [ ]:
print("Loading dataset...")
file_path = 'consolidated_file.csv'
df = pd.read_csv(file_path, low_memory=False)

print(f"Initial dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}\n")

Loading dataset...
Initial dataset shape: (668406, 33)
Columns: ['Time', 'Product', 'ID', 'Top-level category', 'Second-level category', 'Third-level category', 'Fourth-level category', 'Fifth-level category', 'Avg.SKU Price(₱)', 'Brand', 'Shop', 'Sold/Day', 'Revenue/Day(₱)', 'Sold/M', 'Product sales rate(%)', 'Price(₱)', 'Brand ID', 'Link', 'Seller From', 'Joined', 'Listing Time', 'Shop ID', 'SKU', 'Revenue/Month', 'Sold/Month(₱)', 'Sold', 'New Ratings', 'Ratings', 'Likes', 'Ratings Rate', 'Rating Star', 'New likes', 'Unnamed: 31']



# ================================
# 2. Clean Column Names
# ================================

In [ ]:
print("Cleaning column names...")
df.columns = df.columns.str.strip().str.replace(' ', '_').str.lower()
df = df.drop(columns=['unnamed:_31', 'unnamed:_32'], errors='ignore')

print(f"Cleaned columns: {df.columns.tolist()}\n")

Cleaning column names...
Cleaned columns: ['time', 'product', 'id', 'top-level_category', 'second-level_category', 'third-level_category', 'fourth-level_category', 'fifth-level_category', 'avg.sku_price(₱)', 'brand', 'shop', 'sold/day', 'revenue/day(₱)', 'sold/m', 'product_sales_rate(%)', 'price(₱)', 'brand_id', 'link', 'seller_from', 'joined', 'listing_time', 'shop_id', 'sku', 'revenue/month', 'sold/month(₱)', 'sold', 'new_ratings', 'ratings', 'likes', 'ratings_rate', 'rating_star', 'new_likes']



# ================================
# 3. Convert Data Types
# ================================

In [ ]:
# Numeric columns - remove commas and convert
numeric_cols = [
    'avg.sku_price(₱)', 'sold/day', 'revenue/day(₱)', 'sold/m',
    'product_sales_rate(%)', 'price(₱)', 'revenue/month', 'sold/month(₱)',
    'sold', 'new_ratings', 'ratings', 'likes', 'ratings_rate', 
    'rating_star', 'new_likes', 'sku'
]

for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace(',', '').replace('', np.nan)
        df[col] = pd.to_numeric(df[col], errors='coerce')

# Date columns - proper conversion
print("Converting date columns...")

# Time column (YYYYMM format)
df = df.dropna(subset=['time'])  # Drop rows with missing time
df['time'] = df['time'].astype(int).astype(str) + '01'
df['time'] = pd.to_datetime(df['time'], format='%Y%m%d', errors='coerce')

# Listing_time and Joined (YYYYMMDD format)
for col in ['listing_time', 'joined']:
    if col in df.columns:
        df[col] = df[col].astype(str).str.replace('.0', '', regex=False)
        df[col] = pd.to_datetime(df[col], format='%Y%m%d', errors='coerce')

# Convert ID to string
df['id'] = df['id'].fillna(0).astype('Int64').astype(str)

print(f"Date range: {df['time'].min()} to {df['time'].max()}")
print(f"Dataset shape after date conversion: {df.shape}\n")

Converting data types...
Converting date columns...
Date range: 2022-03-01 00:00:00 to 2025-11-01 00:00:00
Dataset shape after date conversion: (668395, 32)



# ================================
# 4. Clean Categories
# ================================

In [ ]:
print("Cleaning category columns...")
category_cols = [
    'top-level_category', 'second-level_category', 'third-level_category',
    'fourth-level_category', 'fifth-level_category'
]

for col in category_cols:
    if col in df.columns:
        df[col] = df[col].fillna('Unknown').str.strip()

Cleaning category columns...


# ================================
# 5. Create Full Monthly Grid per Product
# ================================

In [ ]:
print("Creating full monthly grid per product...")

# Get date range and unique products
all_months = pd.date_range(start=df['time'].min(), 
                           end=df['time'].max(), 
                           freq='MS')
all_products = df['product'].unique()

print(f"Number of unique products: {len(all_products)}")
print(f"Number of months: {len(all_months)}")
print(f"Expected grid size: {len(all_products) * len(all_months):,} rows")

# Create full grid
full_index = pd.MultiIndex.from_product(
    [all_products, all_months], 
    names=['product', 'time']
)
full_df = pd.DataFrame(index=full_index).reset_index()


Filtering top 10 categories by total sales...

Top 10 Categories by Total Sales:
1. Fashion Accessories: 22,060,943,069 units
2. Home & Living: 1,450,792,132 units
3. Beauty: 966,764,633 units
4. Pets: 785,190,542 units
5. Stationery: 379,926,065 units
6. Health: 346,715,773 units
7. Women Clothes: 290,469,337 units
8. Mobile & Gadgets: 278,215,625 units
9. Mom & Baby: 258,078,997 units
10. Sports & Outdoors: 214,831,163 units

Dataset shape after filtering to top 10 categories: (499730, 32)



# ================================
# 6. Merge with Actual Data
# ================================

In [ ]:
print("\nMerging with actual data...")

# Merge all data
full_df = full_df.merge(
    df, 
    on=['product', 'time'], 
    how='left'
)

print(f"Full grid shape: {full_df.shape}")


Creating full monthly grid per product...
Number of unique products: 120853
Number of months: 45
Expected grid size: 5,438,385 rows


# ================================
# 7. Get Product Metadata (One Per Product)
# ================================

In [ ]:
print("Extracting product metadata...")

# Columns that should be constant per product
metadata_cols = [
    'id', 'top-level_category', 'second-level_category', 
    'third-level_category', 'fourth-level_category', 'fifth-level_category',
    'brand', 'brand_id', 'shop', 'shop_id', 'seller_from', 
    'link', 'listing_time', 'joined'
]

# Get first non-null value for each product
product_metadata = df.groupby('product')[metadata_cols].first().reset_index()


Merging with actual data...
Full grid shape: (5459623, 32)



# ================================
# 8. Merge Metadata and Forward-Fill
# ================================

In [ ]:
print("Merging metadata and forward-filling...")

# Drop existing metadata columns to avoid conflicts
full_df = full_df.drop(columns=metadata_cols, errors='ignore')

# Merge metadata
full_df = full_df.merge(product_metadata, on='product', how='left')

# Forward-fill time-varying numeric columns per product
time_varying_cols = [
    'avg.sku_price(₱)', 'price(₱)', 'sku', 
    'ratings', 'rating_star', 'likes'
]

print("Forward-filling time-varying columns...")
for col in time_varying_cols:
    if col in full_df.columns:
        full_df[col] = full_df.groupby('product')[col].ffill()

Extracting product metadata...


# ================================
# 9. Fill Missing Sales/Revenue Based on Listing Time
# ================================

In [ ]:
print("\nFilling missing sales/revenue based on listing_time...")

# Only fill with 0 if time >= listing_time
full_df['sold/m'] = np.where(
    full_df['time'] >= full_df['listing_time'],
    full_df['sold/m'].fillna(0),
    np.nan
)

full_df['revenue/month'] = np.where(
    full_df['time'] >= full_df['listing_time'],
    full_df['revenue/month'].fillna(0),
    np.nan
)

# Fill sold/day and revenue/day similarly
full_df['sold/day'] = np.where(
    full_df['time'] >= full_df['listing_time'],
    full_df['sold/day'].fillna(0),
    np.nan
)

full_df['revenue/day(₱)'] = np.where(
    full_df['time'] >= full_df['listing_time'],
    full_df['revenue/day(₱)'].fillna(0),
    np.nan
)


Merging metadata and forward-filling...
Forward-filling time-varying columns...


# ================================
# Data Quality Checks
# ================================

In [ ]:
print("\nPerforming data quality checks...")

# Check missing values
missing_summary = pd.DataFrame({
    'Missing Count': full_df.isna().sum(),
    'Missing %': (full_df.isna().sum() / len(full_df) * 100).round(2)
}).sort_values(by='Missing %', ascending=False)

print("\nMissing values summary (top 10):")
print(missing_summary.head(10))

# Check for products with no sales data
products_no_sales = full_df.groupby('product')['sold/m'].sum() == 0
print(f"\nProducts with no sales data: {products_no_sales.sum()}")

# Visualize monthly records
print("\nGenerating visualizations...")
monthly_counts = full_df.groupby('time').size()

plt.figure(figsize=(12, 5))
monthly_counts.plot(color='steelblue', linewidth=2)
plt.title('Records per Month (After Grid Creation)', fontsize=14)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Number of Records', fontsize=12)
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('monthly_records.png', dpi=150)
plt.show()


Filling missing sales/revenue based on listing_time...


# ================================
# 11. Save Cleaned Dataset
# ================================

In [ ]:
print("\nSaving cleaned dataset...")

# Sort by product and time
full_df = full_df.sort_values(['product', 'time']).reset_index(drop=True)

# Save to CSV
output_file = 'consolidated_file_cleaned.csv'
full_df.to_csv(output_file, index=False)

print(f"\n{'='*60}")
print(f"PIPELINE COMPLETED SUCCESSFULLY")
print(f"{'='*60}")
print(f"Final dataset shape: {full_df.shape}")
print(f"Date range: {full_df['time'].min()} to {full_df['time'].max()}")
print(f"Number of products: {full_df['product'].nunique()}")
print(f"Number of months: {full_df['time'].nunique()}")
print(f"Output saved to: {output_file}")
print(f"{'='*60}\n")

# Display sample
print("Sample of cleaned data:")
print(full_df[['product', 'time', 'top-level_category', 'sold/m', 'revenue/month']].head(20))


Saving cleaned dataset...

PIPELINE COMPLETED SUCCESSFULLY
Final dataset shape: (5459623, 32)
Date range: 2022-03-01 00:00:00 to 2025-11-01 00:00:00
Number of products: 120853
Number of months: 45
Output saved to: consolidated_file_cleaned_3.csv

Sample of cleaned data:
                                              product       time  \
0      Cute Different Designs  button accessories ... 2022-03-01   
1      Cute Different Designs  button accessories ... 2022-04-01   
2      Cute Different Designs  button accessories ... 2022-05-01   
3      Cute Different Designs  button accessories ... 2022-06-01   
4      Cute Different Designs  button accessories ... 2022-07-01   
5      Cute Different Designs  button accessories ... 2022-08-01   
6      Cute Different Designs  button accessories ... 2022-09-01   
7      Cute Different Designs  button accessories ... 2022-10-01   
8      Cute Different Designs  button accessories ... 2022-11-01   
9      Cute Different Designs  button accessorie